In [ ]:
from can.interface import Bus
import can
from cantools.database import load_file

In [ ]:
import sys
import os
sys.path.append(os.getcwd())
sys.path.append(os.getcwd() + '\\..\\..')
print(sys.path)
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from broqer import Sink
import ipywidgets as widgets

In [ ]:
from canbro import Node
from canbro.e2e import update_e2e_autosar_profile2

In [ ]:
db = load_file('device_CAN_e2e.dbc')
db.nodes

In [ ]:
db.messages[0].dbc.attributes['DataIds'].value

In [ ]:
db.messages[0].signals[0].dbc.attributes['GenSigFuncType'].value

In [ ]:
bus_e= Bus('test', interface='virtual')
ecu = Node(name="ECU",bus=bus_e,database=db )

In [ ]:
bus_v= Bus('test', interface='virtual')
vcu= Node(name='CONTROL',bus=bus_v,database=db )  

In [ ]:
label_value = widgets.Label(value="value=?")
def show_vcu_value(value, label: widgets.Label):
    label.value = 'value={}'.format(value)

In [ ]:
show_print = ecu.DEM._signal_DEM_SNC.subscribe(Sink(show_vcu_value, label=label_value))

In [ ]:
vcu.DEM._signal_operation_mode._metadata.initial

In [ ]:
vcu.DEM._signal_operation_mode._metadata.initial=0

In [ ]:
ecu.DEM._signal_operation_mode.notify(0)

In [ ]:
import ipywidgets as widgets
import time

def update_ecu_value():
    for i in range(6):
        vcu.DEM._signal_operation_mode.notify(i)
        print(f'show Count: {i}')
        time.sleep(20)
    vcu.DEM._signal_operation_mode.notify("Standby")
    print(f'show Count: Standby')

def start_task(button):
    button.description = 'is running'
    update_ecu_value()
    

button = widgets.Button(description='Start')
button.on_click(start_task)

display(button)

In [ ]:
display(label_value)

In [ ]:
def update_E2E( msg: can.Message):
    # Get the message from the database using the arbitration ID
    message = db.get_message_by_frame_id(msg.arbitration_id)

    if "DEM" == message.name:
        update_e2e_autosar_profile2(msg)
        vcu.DEM._signal_DEM_CRC.notify(msg.data[0])
        vcu.DEM._signal_DEM_SNC.notify(msg.data[1] & 0x0F)

In [ ]:
vcu.DEM.start_periodic(update_E2E)

In [ ]:
vcu.DEM._signal_operation_mode.notify(3)

In [ ]:
type(vcu.DEM)

In [ ]:
type(ecu.DEM)